在第3章介绍的三个例子中，我们将数据划分为训练集、验证集和测试集。我们没有在训练模型的相同数据上对模型进行评估，其原因很快显而易见：仅仅几轮过后，三个模型都开始过拟合。也就是说，随着训练的进行，模型在训练数据上的性能始终在提高，但在前所未见的数据上的性能则不再变化或者开始下降。

机器学习的目的是得到可以泛化（generalize）的模型，即在前所未见的数据上表现很好的模型，  

而过拟合则是核心难点。  

你只能控制可以观察的事情，所以能够可靠地衡量模型的泛化能力非常重要。后面几节将介绍降低过拟合以及将泛化能力最大化的方法。本节重点介绍如何衡量泛化能力，即如何评估机器学习模型

### 4.2.1　训练集、验证集和测试集
评估模型的重点是将数据划分为三个集合：训练集、验证集和测试集。在训练数据上训练模型，在验证数据上评估模型。一旦找到了最佳参数，就在测试数据上最后测试一次。  

不能只是一个训练集和一个测试集，在训练集上训练模型，然后在测试集上评估模型，这样常常会过拟合。造成这一现象的关键在于信息泄露（information  leak）。每次基于模型在验证集上的性能来调节模型超参数，都会有一些关于验证数据的信息泄露到模型中。如果对每个参数只调节一次，那么泄露的信息很少，验证集仍然可以可靠地评估模型。但如果你多次重复这一过程（运行一次实验，在验证集上评估，然后据此修改模型），那么将会有越来越多的关于验证集的信息泄露到模型中。  

数据划分为训练集、验证集和测试集可能看起来很简单，但如果可用数据很少，还有几种高级方法可以派上用场。我们先来介绍三种经典的评估方法：  
简单的留出验证;  
K折验证;  
带有打乱数据的重复K折验证。

#### 4.2.1.1 简单的留出验证
留出一定比例的数据作为测试集。在剩余的数据上训练模型，然后在测试集上评估模型。  
因此最后还需要一个验证集

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import matplotlib.pyplot as plt

In [ ]:
num_validation_sample=10000
np.random.shuffle(data) # 这一步用于打乱数据

validation_data=data[:num_validation_sample] # 验证集
data=data[num_validation_sample:]
training_data=data #训练集

model=get_model() # 在训练集上训练模型
model.train(training_data)
validation_score=model.evaluate(validation_data) # 在验证集上评估模型

.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节
.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节
.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节



model=get_model() # 得到最终调节好的超参数
model.train(np.concatenate(training_data, #将train & validation 数据合并，进行整体训练
                          validation_data)
test_score=model.evaluate(test_data) # 在验证集上评估模型


缺点：  
1. 如果可用的数据很少，那么可能验证集和测试集包含的样本就太少，从而无法在统计学上代表数据。  
2. 如果在划分数据前进行不同的随机打乱，最终得到的模型性能差别很大。

#### 4.2.1.2 K折验证
K折验证（K-fold validation）将数据划分为大小相同的K个分区。对于每个分区i，在剩余的K-1个分区上训练模型，然后在分区i上评估模型。最终分数等于K个分数的平均值。对于不同的训练集 – 测试集划分，如果模型性能的变化很大，那么这种方法很有用。  
与留出验证一样，这种方法也需要独立的验证集进行模型校正

In [ ]:
k = 4
num_validation_samples = len(data) // k # // 取整数
np.random.shuffle(data)
validation_scores = []
for i in range(k):
    print(f"Processing fold #{i}") 
    validation_data=data[num_validation_samples * i : num_validation_samples * (i+1)] # 准备验证数据
    
    training_data=data[:num_validation_samples * i]+ # 验证数据之外的数据做训练数据
    data[num_validation_samples * (i+1):]
   
    
    model = build_model() # 构建模型
    model.train(training_data)
    validation_score=model.evaluate(validation_data)
    validation_scores.append(validation_score) # 添加验证分数

validation_score=np.average(validation_scores) # 取K重验证的平均数 

.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节
.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节
.# 根据验证集的评估结果调节参数，重新训练，重新评估，然后再次调节

model=get_model() # 得到最终调节好的超参数
model.train(np.concatenate(training_data, #将train & validation 数据合并，进行整体训练
                          validation_data)
test_score=model.evaluate(test_data) # 在验证集上评估模型


#### 4.2.1.3 带有打乱数据的重复K折验证
如果可用的数据相对较少，而你又需要尽可能精确地评估模型，那么可以选择带有打乱数据的重复 K折验证（iterated  K-fold validation  with shuffling）。我发现这种方法在 Kaggle竞赛中特别有用。具体做法是多次使用K折验证，在每次将数据分为K个分区之前都先将数据打乱。最终分数是每次  K折验证分数的平均值。注意，这种方法一共要训练和评估P×K个模型（P是重复次数），计算代价很大

### 4.2.2　评估模型的注意事项
数据代表性（data   representativeness）。你希望训练集和测试集都能够代表当前数据。例如，你想要对数字图像进行分类，而图像样本是按类别排序的，如果你将前 80%作为训练集，剩余 20%作为测试集，那么会导致训练集中只包含类别   0~7，而测试集中只包含类别 8~9。这个错误看起来很可笑，却很常见。因此，在将数据划分为训练集和测试集之前，通常应该随机打乱数据。  

时间箭头（the  arrow of time）。如果想要根据过去预测未来（比如明天的天气、股票走势等），那么在划分数据前你不应该随机打乱数据，因为这么做会造成时间泄露（temporalleak）：你的模型将在未来数据上得到有效训练。在这种情况下，你应该始终确保测试集中所有数据的时间都晚于训练集数据。  

数据冗余（redundancy   in your data）。如果数据中的某些数据点出现了两次（这在现实中的数据里十分常见），那么打乱数据并划分成训练集和验证集会导致训练集和验证集之间的数据冗余。从效果上来看，你是在部分训练数据上评估模型，这是极其糟糕的！一定要确保训练集和验证集之间没有交集